We're going to use the Generator and discriminators to thus train on the LJSpeech dataset. The dataset consists of around 13,000 samples reading from 7 non-fiction books. They vary from 1 - 10 seconds and total around 24h.  



In [4]:
import argparse
import jax
import os
import librosa
import optax
import wandb
from tqdm import tqdm
import equinox as eqx

from Generator import Generator
from Discriminators import MultiPeriodDiscriminator, MultiScaleDiscriminator, feature_loss, generator_loss, discriminator_loss

def create_parser():
    parser = argparse.ArgumentParser(description="Arguments for training HiFiGaN")

    parser.add_argument("--dataset_path", "Path to the dataset to use (LJSpeech)")

    parser.add_argument("--learning_rate", "Learning rate during training")

    parser.add_argument("--output_path", "Path to store model weights")

    return parser

def save_model(model, path):
    eqx.tree_serialise_leaves(path, model)

def get_dataset(dataset_path):
    mel_dir = os.path.join(dataset_path, 'mel_spectrograms')
    wav_dir = os.path.join(dataset_path, 'processed_wavs')

    mels = []
    wavs = []

    for filename in os.listdir(mel_dir):
        np_data = jax.numpy.load(os.path.join(mel_dir, filename))
        # print(np_data.shape)
        mels.append(np_data[:,:-1])

    for filename in os.listdir(wav_dir):
        wav_data, _ = librosa.load(os.path.join(wav_dir, filename[:-4]+".mp3"))
        wav_data = jax.numpy.array(wav_data)
        wav_data = jax.numpy.expand_dims(wav_data, 0)
        wavs.append(wav_data)


    return jax.numpy.array(mels), jax.numpy.array(wavs)



@eqx.filter_value_and_grad
def calculate_gan_loss(gan, period, scale, x, y):

    gan_result = jax.vmap(gan)(x)
    fake_scale, _, _, _ = jax.vmap(scale)(gan_result, y)
    fake_period, _, _, _ = jax.vmap(period)(gan_result, y)

    l1_loss = jax.numpy.mean(jax.numpy.abs(gan_result - y)) # L1 loss
    G_loss = 0
    for fake in fake_period:
        G_loss += jax.numpy.mean((fake - 1) ** 2)
    for fake in fake_scale:
        G_loss += jax.numpy.mean((fake - 1) ** 2)
    # G_loss_scale = jax.numpy.mean((fake_scale - jax.numpy.ones(batch_size)) ** 2)

    return G_loss + 30 * l1_loss

@eqx.filter_value_and_grad
def calculate_disc_loss(model, fake, real):
    fake_result, real_result, _, _ = jax.vmap(model)(fake, real)
    loss = 0
    for fake_res, real_res in zip(fake_result, real_result):
        fake_loss = jax.numpy.mean((fake_res)**2)
        real_loss = jax.numpy.mean((real_res-1)**2)
        loss += (fake_loss + real_loss)

    return loss

@eqx.filter_jit
def make_step(gan, period_disc, scale_disc, x, y, gan_optim, period_optim, scale_optim, optim1, optim2, optim3):

    result = jax.vmap(gan)(x)

    trainable_scale, _ = eqx.partition(scale_disc, eqx.is_inexact_array)
    trainable_period, _ = eqx.partition(period_disc, eqx.is_inexact_array)

    loss_scale, grads_scale = calculate_disc_loss(scale_disc, result, y)
    updates, scale_optim = optim2.update(grads_scale, scale_optim)
    scale_disc = eqx.apply_updates(trainable_scale, updates)

    loss_period, grads_period = calculate_disc_loss(period_disc, result, y)
    updates, period_optim = optim3.update(grads_period, period_optim)
    period_disc = eqx.apply_updates(trainable_period, updates)

    loss_gan, grads_gan = calculate_gan_loss(gan, period_disc, scale_disc, x, y)
    updates, gan_optim = optim1.update(grads_gan, gan_optim)
    gan = eqx.apply_updates(gan, updates)
    
    return loss_gan, loss_period, loss_scale, gan, period_disc, scale_disc, gan_optim, period_optim, scale_optim

def train_hifigan(dataset_path, output_path, learning_rate=1e-4, batch_size=32, epochs=100, seed=69):
    run = wandb.init(
    # Set the project where this run will be logged
    project="HiFiGaN JAX",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": learning_rate,
        "epochs": epochs,
        "batch_size": batch_size,
        "PRNG_SEED": seed,
    },
)

    key = jax.random.PRNGKey(seed)

    key1, key2, key3 = jax.random.split(key, 3)

    generator = Generator(channels_in=80, channels_out=1, key=key1)
    generator = eqx.tree_deserialise_leaves("checkpoints_trained_to_100/generator_epoch_90.eqx", like=generator) 
    scale_disc = MultiScaleDiscriminator(key=key2)
    period_disc = MultiPeriodDiscriminator(key=key3)

    mels, wavs = get_dataset(dataset_path)


    optim1 = optax.adam(2e-4, b1=0.8, b2=0.99)
    gan_optim = optim1.init(generator)
    
    optim2 = optax.adam(learning_rate)
    trainable_scale, _ = eqx.partition(scale_disc, eqx.is_inexact_array)
    scale_optim = optim2.init(trainable_scale)

    optim3= optax.adam(learning_rate)
    trainable_period, _ = eqx.partition(period_disc, eqx.is_inexact_array)
    period_optim = optim3.init(trainable_period)

    dataset_size = len(wavs)

    for epoch in tqdm(range(epochs)):
        avg_gan_loss = 0
        avg_scale_loss = 0
        avg_period_loss = 0

        key, subkey = jax.random.split(key)
        perm = jax.random.permutation(subkey, dataset_size)
        
        for batch_start in range(0, dataset_size, batch_size):
            batch_indices = perm[batch_start: batch_start + batch_size]
            x = mels.take(batch_indices, axis=0)
            y = wavs.take(batch_indices, axis=0)
            
            # Display batch indices and data
            gan_loss, period_loss,scale_loss, generator, period_disc, scale_disc, gan_optim, period_optim, scale_optim = make_step(generator, period_disc, scale_disc, x, y, gan_optim, period_optim,scale_optim, optim1, optim2, optim3)
            # print(grads)        loss = loss.item()
            wandb.log({"GAN loss": gan_loss, "SCALE loss": scale_loss, "PERIOD loss": period_loss})

            avg_gan_loss += gan_loss
            # avg_scale_loss += scale_loss
            avg_period_loss += period_loss
            # print(batch_data.shape)

            # print(res.shape)

        if epoch % 10 == 0: 
            save_model(generator, os.path.join(output_path, f"generator_epoch_{epoch}.eqx"))
        print(f"Average gan loss: {avg_gan_loss/dataset_size}, Avg scale loss: {avg_scale_loss/dataset_size}, Avg period loss: {avg_period_loss/dataset_size}")

# if __name__ == "__main__":
#     parser = create_parser()
#     args = parser.parse_args()

#     train_hifigan(dataset_path=args.dataset_path, output_path=args.output_path, learning_rate=args.learning_rate)


In [5]:
train_hifigan(dataset_path="dataset", output_path="checkpoint", learning_rate=1e-5)

  0%|          | 0/100 [00:00<?, ?it/s]2024-10-27 15:35:45.840517: W external/xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 33.03GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-10-27 15:35:46.820631: W external/xla/xla/tsl/framework/bfc_allocator.cc:306] Allocator (GPU_0_bfc) ran out of memory trying to allocate 33.03GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
  1%|          | 1/100 [01:56<3:12:40, 116.77s/it]

Average gan loss: 0.19403529167175293, Avg scale loss: 0.0, Avg period loss: 0.07992879301309586


  2%|▏         | 2/100 [03:11<2:30:48, 92.33s/it] 

Average gan loss: 0.14877063035964966, Avg scale loss: 0.0, Avg period loss: 0.07096628844738007


  3%|▎         | 3/100 [04:27<2:16:39, 84.53s/it]

...

Average gan loss: 0.2242295742034912, Avg scale loss: 0.0, Avg period loss: 0.03231711685657501


 99%|█████████▉| 99/100 [2:05:04<01:15, 75.39s/it]

Average gan loss: 0.22289864718914032, Avg scale loss: 0.0, Avg period loss: 0.032436124980449677


100%|██████████| 100/100 [2:06:19<00:00, 75.79s/it]

Average gan loss: 0.2256462723016739, Avg scale loss: 0.0, Avg period loss: 0.03154291212558746


In [ ]:
import jax
from Generator import Generator 

key1 = jax.random.PRNGKey(69)
generator = Generator(channels_in=80, channels_out=1, key=key1)
print(generator)
